# Gene expression exploration of relevant receptors in male mice treated with AAV9

In this script, we first explore the spatial gene expression maps of some receptros that, according to the literature, could be of potential interest for the internalization of the AAV. Then, we will use [LIANA+](https://www.biorxiv.org/content/10.1101/2023.08.19.553863v1) and their spatially-informed bivariate metrics to identify pairs of co-expressed receptors with the transgene. In particular, we will focus on males treated with AAV9 in this script. 

In [ ]:
import pandas as pd
import scanpy as sc
import plotnine as p9
import liana as li
import os
import anndata as ad
import numpy as np
from scipy.stats import combine_pvalues
from matplotlib import pyplot as plt
import seaborn as sns

from mudata import MuData

In [ ]:
def combine_row_pvalues(row):
    # Filter out NaN values if necessary
    p_values = row.dropna()
    # Apply Fisher's method to combine p-values
    _, combined_p = combine_pvalues(p_values, method='fisher')
    return combined_p

In [ ]:
# Automatically re-load wrapper functions after an update
# Find details here: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
root_path = os.getcwd()
inpath='your_inpath_folder' # Replace with the location of your samples
results_folder = os.path.join(root_path, 'analyzed', 'zonation')

We read our pre-processed anndata objects and we select the males treated with AAV9 

In [ ]:
file_names = [f for f in os.listdir(results_folder) if os.path.isfile(os.path.join(results_folder, f))]

adata_list = [ad.read(os.path.join(results_folder, file)) for file in file_names if file.endswith('.h5ad')]

## Exploration of the Spatial distrbution of the Expression of some relevant receptors

We first, explore the expression of some receptors that according to the literature are relevant for the internalization of the AAV. 

In [ ]:
receptors_set = [
    'Met',
    'AU040320',
    'Fgfr1',
    'Hspg2',
    'Rpsa',
    'Cd9',
    'Itgb5',
    'Itgav',
    'Itgb1',
] 

## Spatially-informed Bivariate Metrics

Here, we will use [LIANA+](https://www.biorxiv.org/content/10.1101/2023.08.19.553863v1) to compute spatially informed local scores that in order to help identifying receptors that are spatially co-expressed with the transgene. and therefore are more likely to be involved in the internalization of the AAV. In particular, we will use spatially informed cosine similarity. We extracted the list of receptors from [CellCommuNet](https://academic.oup.com/nar/advance-article/doi/10.1093/nar/gkad906/7321072?login=true). To get receptors that are relevant in mice liver, we filtered by mus musculus, normal condition, study type single, and tissue liver. Of note, the complexes have human nomenclature,so I will drop them. We also extracted mouse genes from uniprot that are annotated as SL-0039 as subcellular location term.  

In [ ]:
df_cell_interactions = pd.read_csv('CellCommResults.csv')
df_cell_interactions

In [ ]:
Uniprot_receptors_df =  pd.read_csv("uniprotkb_cc_scl_term_SL_0039_AND_model_2024_04_22.tsv", sep="\t")
Uniprot_receptors = Uniprot_receptors_df['Gene Names'].unique().tolist()
Uniprot_receptors = [record for record in Uniprot_receptors if isinstance(record, str)]
## There are many records that contain more than one gene. I trey to split them: 
Uniprot_receptors_individual = [gene for record in Uniprot_receptors for gene in record.split()]

In [ ]:
receptors = df_cell_interactions['Receptor'].unique().tolist() + receptors_set + Uniprot_receptors_individual

In [ ]:
receptors = list(set(receptors))
len(receptors)

This is the list of spatially informed bivarite metrics. As described above, we will used the spatially weigthed cosine similarity. 

In [ ]:
li.method.bivar.show_functions()

In [ ]:
plot, _ = li.ut.query_bandwidth(coordinates=adata_list[0].obsm['spatial'], start=0, end=500, interval_n=20)
plot + p9.scale_y_continuous(breaks=range(0, 500, 5))

We generate our own data frame of ligand receptor interactions. The ligand is also the transgene and the receptors are the ones described above

In [ ]:
df_ligand_receptor_int = pd.DataFrame({'ligand': 'cisAAV-CMV-GFP-WPRE', 'receptor' : receptors})

In [ ]:
df_ligand_receptor_int

## Males AAV9

In [ ]:
adata_list_males_AVV9 = []

for adata in adata_list:
    
    if adata.obs['Gender'].unique()[0] == 'Male' and adata.obs['Condition'].unique()[0] == 'AAV9-CMV-GFP': 
        adata_list_males_AVV9.append(adata)        

In [ ]:
adata_list_males_AVV9[0].obs

In [ ]:
from itertools import product
result_list_adata = []

for adata in adata_list_males_AVV9: 
    
    
    li.ut.spatial_neighbors(adata, bandwidth=150, cutoff=0.1, kernel='gaussian', set_diag=True)
    
    li.mt.lr_bivar(adata,
               function_name='morans', # Name of the function
               resource=df_ligand_receptor_int,
               n_perms=1000, # Number of permutations to calculate a p-value
               mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
               add_categories=True, # Whether to add local categories to the results
               expr_prop=0.2, # Minimum expr. proportion for ligands/receptors and their subunits
               use_raw=False,
               verbose=True)
    
    result_list_adata.append(adata)

In [ ]:
## Compute corrected p-values 
result_list_adata[0].obsm['local_scores'].var.sort_values(by='morans_r', ascending=False).head(35)
# result_list_adata[0].obsm['local_scores'].var.sort_values(by='mean', ascending=False).head(35)

In [ ]:
result_df = result_list_adata[0].obsm['local_scores'].var
for current_adata in result_list_adata[1:]: 
    current_df = current_adata.obsm['local_scores'].var
    result_df = result_df.merge(current_df, on='interaction', how='inner')

In [ ]:
average_morans = result_df[['morans_r', 'morans_r_x', 'morans_r_x']].mean(axis=1)

In [ ]:
combined_pvalues = result_df[['morans_pvals', 'morans_pvals_x', 'morans_pvals_y']].apply(combine_row_pvalues, axis=1)

In [ ]:
final_df = pd.DataFrame({
    'ligand': result_df['ligand'],
    'receptor': result_df['receptor'],
    'interaction': result_df.index, 
    'average_morans': average_morans, 
    'abs_average_morans': abs(average_morans),
    'combined_pvalues': combined_pvalues
   })

In [ ]:
final_df.sort_values(by='abs_average_morans', ascending=False).head(35)

In [ ]:
## Filtering by thresholds. 
pvalue_cutoff = 0.01
# Morans_cutoff = 0.1
Morans_cutoff = 0.025

filtered_final_df = final_df[(final_df['abs_average_morans'] > Morans_cutoff) & (final_df['combined_pvalues'] < pvalue_cutoff)]

filtered_final_df.to_csv('LianaResults/Males_AAV9_cc_scl_SL0039.csv', index=False)

filtered_final_sorted_df_top20 = filtered_final_df.sort_values(by='average_morans', ascending=False).head(10)
filtered_final_sorted_df_top20 = filtered_final_sorted_df_top20.sort_values('average_morans', ascending=True)

In [ ]:
colors = ['blue' if x < 0 else 'red' for x in filtered_final_sorted_df_top20['average_morans']]

# Create the bar plot
plt.figure(figsize=(8, 8))
plt.barh(filtered_final_sorted_df_top20['receptor'], filtered_final_sorted_df_top20['average_morans'], color = colors)

# Add labels and title if desired
plt.xlabel('Average Moran\'s I')
plt.ylabel('Receptor')
plt.title('Bar Plot of Average Moran\'s I by Receptor')

# Display the plot
plt.show()

In [ ]:
top_interactions = filtered_final_df.sort_values('average_morans', ascending=False).head(2).index
top_receptors = filtered_final_df.sort_values('average_morans', ascending=False).head(2)['receptor'].tolist()
### Adding plots requested by Bettina
interactions_to_check = ['cisAAV-CMV-GFP-WPRE^Rpsa', 'cisAAV-CMV-GFP-WPRE^Slco1b2', 'cisAAV-CMV-GFP-WPRE^Pigr', 'cisAAV-CMV-GFP-WPRE^Fbp1']
receptors_to_check = ['Rpsa','Slco1b2','Pigr','Fbp1']

for current_adata in result_list_adata:
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=top_interactions, size=1.25, vmax=1, vmin=-1, cmap='coolwarm')
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=top_interactions, size=1.25, cmap="magma_r")
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=top_interactions, size=1.25, cmap="coolwarm")

    ## Aditional plots
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=interactions_to_check, size=1.25, vmax=1, vmin=-1, cmap='coolwarm', ncols= 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=interactions_to_check, size=1.25, cmap="magma_r", ncols = 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=interactions_to_check, size=1.25, cmap="coolwarm", ncols = 2)
    
    
for current_adata in adata_list_males_AVV9: 
    
    sc.pl.spatial(current_adata, color =top_receptors, size=1.25)

    ## Aditional plots

    sc.pl.spatial(current_adata, color =receptors_to_check, size=1.25, ncols =2 )


## Males AAV2

In [ ]:
adata_list_males_AVV2 = []

for adata in adata_list:
    
    if adata.obs['Gender'].unique()[0] == 'Male' and adata.obs['Condition'].unique()[0] == 'AAV2-CMV-GFP': 
        adata_list_males_AVV2.append(adata)        

In [ ]:
adata_list_males_AVV2[0].obs

In [ ]:
from itertools import product
result_list_adata = []

for adata in adata_list_males_AVV2: 
    
    
    li.ut.spatial_neighbors(adata, bandwidth=150, cutoff=0.1, kernel='gaussian', set_diag=True)
    
    li.mt.lr_bivar(adata,
               function_name='morans', # Name of the function
               resource=df_ligand_receptor_int,
               n_perms=1000, # Number of permutations to calculate a p-value
               mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
               add_categories=True, # Whether to add local categories to the results
               expr_prop=0.2, # Minimum expr. proportion for ligands/receptors and their subunits
               use_raw=False,
               verbose=True)
    
    result_list_adata.append(adata)

In [ ]:
## Compute corrected p-values 
result_list_adata[0].obsm['local_scores'].var.sort_values(by='morans_r', ascending=False).head(35)

In [ ]:
result_df = result_list_adata[0].obsm['local_scores'].var
for current_adata in result_list_adata[1:]: 
    current_df = current_adata.obsm['local_scores'].var
    result_df = result_df.merge(current_df, on='interaction', how='inner')

In [ ]:
average_morans = result_df[['morans_r', 'morans_r_x', 'morans_r_x']].mean(axis=1)

In [ ]:
combined_pvalues = result_df[['morans_pvals', 'morans_pvals_x', 'morans_pvals_y']].apply(combine_row_pvalues, axis=1)

In [ ]:
final_df = pd.DataFrame({
    'ligand': result_df['ligand'],
    'receptor': result_df['receptor'],
    'interaction': result_df.index, 
    'average_morans': average_morans, 
    'abs_average_morans': abs(average_morans),
    'combined_pvalues': combined_pvalues
   })

In [ ]:
final_df.sort_values(by='abs_average_morans', ascending=False).head(35)

In [ ]:
## Filtering by thresholds. 
pvalue_cutoff = 0.01
Morans_cutoff = 0.025

filtered_final_df = final_df[(final_df['abs_average_morans'] > Morans_cutoff) & (final_df['combined_pvalues'] < pvalue_cutoff)]

filtered_final_df.to_csv('LianaResults/Males_AAV2_cc_scl_SL0039.csv', index=False)

filtered_final_sorted_df_top20 = filtered_final_df.sort_values(by='average_morans', ascending=False).head(10)
filtered_final_sorted_df_top20 = filtered_final_sorted_df_top20.sort_values('average_morans', ascending=True)

In [ ]:
colors = ['blue' if x < 0 else 'red' for x in filtered_final_sorted_df_top20['average_morans']]

# Create the bar plot
plt.figure(figsize=(8, 8))
plt.barh(filtered_final_sorted_df_top20['receptor'], filtered_final_sorted_df_top20['average_morans'], color=colors)

# Add labels and title if desired
plt.xlabel('Average Moran\'s I')
plt.ylabel('Receptor')
plt.title('Bar Plot of Average Moran\'s I by Receptor')

# Display the plot
plt.show()

In [ ]:
top_interactions = filtered_final_df.sort_values('average_morans', ascending=False).head(2).index
top_receptors = filtered_final_df.sort_values('average_morans', ascending=False).head(2)['receptor'].tolist()

for current_adata in result_list_adata:
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=top_interactions, size=1.25, vmax=1, vmin=-1, cmap='coolwarm')
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=top_interactions, size=1.25, cmap="magma_r")
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=top_interactions, size=1.25, cmap="coolwarm")

    ## Aditional plots
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=interactions_to_check, size=1.25, vmax=1, vmin=-1, cmap='coolwarm', ncols= 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=interactions_to_check, size=1.25, cmap="magma_r", ncols = 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=interactions_to_check, size=1.25, cmap="coolwarm", ncols = 2)
    
for current_adata in adata_list_males_AVV2: 
    
    sc.pl.spatial(current_adata, color =top_receptors, size=1.25)

    ## Aditional plots

    sc.pl.spatial(current_adata, color =receptors_to_check, size=1.25, ncols =2 )

## Females AAV9

In [ ]:
adata_list_females_AVV9 = []

for adata in adata_list:
    
    if adata.obs['Gender'].unique()[0] == 'Female' and adata.obs['Condition'].unique()[0] == 'AAV9-CMV-GFP': 
        adata_list_females_AVV9.append(adata)        

In [ ]:
adata_list_females_AVV9[0].obs

In [ ]:
from itertools import product
result_list_adata = []

for adata in adata_list_females_AVV9: 
    
    
    li.ut.spatial_neighbors(adata, bandwidth=150, cutoff=0.1, kernel='gaussian', set_diag=True)
    
    li.mt.lr_bivar(adata,
               function_name='morans', # Name of the function
               resource=df_ligand_receptor_int,
               n_perms=1000, # Number of permutations to calculate a p-value
               mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
               add_categories=True, # Whether to add local categories to the results
               expr_prop=0.2, # Minimum expr. proportion for ligands/receptors and their subunits
               use_raw=False,
               verbose=True)
    
    result_list_adata.append(adata)

In [ ]:
## Compute corrected p-values 
result_list_adata[0].obsm['local_scores'].var.sort_values(by='morans_r', ascending=False).head(35)

In [ ]:
result_df = result_list_adata[0].obsm['local_scores'].var
for current_adata in result_list_adata[1:]: 
    current_df = current_adata.obsm['local_scores'].var
    result_df = result_df.merge(current_df, on='interaction', how='inner')

In [ ]:
average_morans = result_df[['morans_r', 'morans_r_x', 'morans_r_x']].mean(axis=1)

In [ ]:
combined_pvalues = result_df[['morans_pvals', 'morans_pvals_x', 'morans_pvals_y']].apply(combine_row_pvalues, axis=1)

In [ ]:
final_df = pd.DataFrame({
    'ligand': result_df['ligand'],
    'receptor': result_df['receptor'],
    'interaction': result_df.index, 
    'average_morans': average_morans, 
    'abs_average_morans': abs(average_morans),
    'combined_pvalues': combined_pvalues
   })

In [ ]:
final_df.sort_values(by='abs_average_morans', ascending=False).head(35)

In [ ]:
## Filtering by thresholds. 
pvalue_cutoff = 0.01
# Morans_cutoff = 0.1
Morans_cutoff = 0.025

filtered_final_df = final_df[(final_df['abs_average_morans'] > Morans_cutoff) & (final_df['combined_pvalues'] < pvalue_cutoff)]

filtered_final_df.to_csv('LianaResults/Females_AAV9_cc_scl_SL0039.csv', index=False)

filtered_final_sorted_df_top20 = filtered_final_df.sort_values(by='average_morans', ascending=False).head(10)
filtered_final_sorted_df_top20 = filtered_final_sorted_df_top20.sort_values('average_morans', ascending=True)

In [ ]:
colors = ['blue' if x < 0 else 'red' for x in filtered_final_sorted_df_top20['average_morans']]

# Create the bar plot
plt.figure(figsize=(8, 8))
plt.barh(filtered_final_sorted_df_top20['receptor'], filtered_final_sorted_df_top20['average_morans'], color=colors)

# Add labels and title if desired
plt.xlabel('Average Moran\'s I')
plt.ylabel('Receptor')
plt.title('Bar Plot of Average Moran\'s I by Receptor')

# Display the plot
plt.show()

In [ ]:
top_interactions = filtered_final_df.sort_values('average_morans', ascending=False).head(2).index
top_receptors = filtered_final_df.sort_values('average_morans', ascending=False).head(2)['receptor'].tolist()

for current_adata in result_list_adata:
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=top_interactions, size=1.25, vmax=1, vmin=-1, cmap='coolwarm')
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=top_interactions, size=1.25, cmap="magma_r")
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=top_interactions, size=1.25, cmap="coolwarm")

    ## Aditional plots
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=interactions_to_check, size=1.25, vmax=1, vmin=-1, cmap='coolwarm', ncols= 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=interactions_to_check, size=1.25, cmap="magma_r", ncols = 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=interactions_to_check, size=1.25, cmap="coolwarm", ncols = 2)
    
for current_adata in adata_list_females_AVV9: 
    
    sc.pl.spatial(current_adata, color =top_receptors, size=1.25)

    ## Aditional plots

    sc.pl.spatial(current_adata, color =receptors_to_check, size=1.25, ncols =2 )

## Females AAV2

In [ ]:
adata_list_females_AVV2 = []

for adata in adata_list:
    
    if adata.obs['Gender'].unique()[0] == 'Female' and adata.obs['Condition'].unique()[0] == 'AAV2-CMV-GFP': 
        adata_list_females_AVV2.append(adata)        

In [ ]:
adata_list_females_AVV2[1].obs

In [ ]:
from itertools import product
result_list_adata = []

for adata in adata_list_females_AVV2: 
    
    
    li.ut.spatial_neighbors(adata, bandwidth=150, cutoff=0.1, kernel='gaussian', set_diag=True)
    
    li.mt.lr_bivar(adata,
               function_name='morans', # Name of the function
               resource=df_ligand_receptor_int,
               n_perms=1000, # Number of permutations to calculate a p-value
               mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
               add_categories=True, # Whether to add local categories to the results
               expr_prop=0.1, # Minimum expr. proportion for ligands/receptors and their subunits
               use_raw=False,
               verbose=True)
    
    result_list_adata.append(adata)

In [ ]:
## Compute corrected p-values 
result_list_adata[0].obsm['local_scores'].var.sort_values(by='morans_r', ascending=False).head(35)

In [ ]:
result_df = result_list_adata[0].obsm['local_scores'].var
for current_adata in result_list_adata[1:]: 
    current_df = current_adata.obsm['local_scores'].var
    result_df = result_df.merge(current_df, on='interaction', how='inner')

In [ ]:
average_morans = result_df[['morans_r', 'morans_r_x', 'morans_r_x']].mean(axis=1)

In [ ]:
combined_pvalues = result_df[['morans_pvals', 'morans_pvals_x', 'morans_pvals_y']].apply(combine_row_pvalues, axis=1)

In [ ]:
final_df = pd.DataFrame({
    'ligand': result_df['ligand'],
    'receptor': result_df['receptor'],
    'interaction': result_df.index, 
    'average_morans': average_morans, 
    'abs_average_morans': abs(average_morans),
    'combined_pvalues': combined_pvalues
   })

In [ ]:
final_df.sort_values(by='abs_average_morans', ascending=False).head(35)

In [ ]:
## Filtering by thresholds. 
pvalue_cutoff = 0.01
# Morans_cutoff = 0.1
Morans_cutoff = 0.025

filtered_final_df = final_df[(final_df['abs_average_morans'] > Morans_cutoff) & (final_df['combined_pvalues'] < pvalue_cutoff)]

filtered_final_df.to_csv('LianaResults/Females_AAV2_cc_scl_SL0039.csv', index=False)

filtered_final_sorted_df_top20 = filtered_final_df.sort_values(by='average_morans', ascending=False).head(10)
filtered_final_sorted_df_top20 = filtered_final_sorted_df_top20.sort_values('average_morans', ascending=True)


In [ ]:
colors = ['blue' if x < 0 else 'red' for x in filtered_final_sorted_df_top20['average_morans']]

# Create the bar plot
plt.figure(figsize=(8, 8))
plt.barh(filtered_final_sorted_df_top20['receptor'], filtered_final_sorted_df_top20['average_morans'], color=colors)

# Add labels and title if desired
plt.xlabel('Average Moran\'s I')
plt.ylabel('Receptor')
plt.title('Bar Plot of Average Moran\'s I by Receptor')

# Display the plot
plt.show()

In [ ]:
top_interactions = filtered_final_df.sort_values('average_morans', ascending=False).head(2).index
top_receptors = filtered_final_df.sort_values('average_morans', ascending=False).head(2)['receptor'].tolist()


for current_adata in result_list_adata:
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=top_interactions, size=1.25, vmax=1, vmin=-1, cmap='coolwarm')
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=top_interactions, size=1.25, cmap="magma_r")
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=top_interactions, size=1.25, cmap="coolwarm")

    ## Aditional plots
    
    sc.pl.spatial(current_adata.obsm['local_scores'], color=interactions_to_check, size=1.25, vmax=1, vmin=-1, cmap='coolwarm', ncols= 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='pvals', color=interactions_to_check, size=1.25, cmap="magma_r", ncols = 2)
    sc.pl.spatial(current_adata.obsm['local_scores'], layer='cats', color=interactions_to_check, size=1.25, cmap="coolwarm", ncols = 2)
    
for current_adata in adata_list_females_AVV9: 
    
    sc.pl.spatial(current_adata, color =top_receptors, size=1.25)

    ## Aditional plots

    sc.pl.spatial(current_adata, color =receptors_to_check, size=1.25, ncols =2 )

In [ ]:
! jupyter nbconvert --to html 15_SpatialConnectivity_Receptors_LianaPvalues_cc_scl_SL0039.ipynb